In [5]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [7]:
X_train_path= os.path.join("..", "data","original_input", "X_train_filled.csv")
X_train_preprocessed = pd.read_csv(X_train_path, delimiter=',')
X_test_path= os.path.join("..", "data","original_input", "X_test_GgyECq8.csv")
X_test_preprocessed = pd.read_csv(X_test_path, delimiter=',')
y_train_path= os.path.join("..", "data","original_input", "y_train_jJtXgMX.csv")
y_train_preprocessed = pd.read_csv(y_train_path, delimiter=',')

X_train_preprocessed

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
0,2022-01-01 01:00:00+00:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,NaN
1,2022-01-01 02:00:00+00:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,NaN
2,2022-01-01 03:00:00+00:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,NaN
3,2022-01-01 04:00:00+00:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,NaN
4,2022-01-01 05:00:00+00:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...
10600,2023-03-29 17:00:00+00:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.11
10601,2023-03-29 18:00:00+00:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.66
10602,2023-03-29 19:00:00+00:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.01
10603,2023-03-29 20:00:00+00:00,47967.0,3386.0,11952.0,38320.0,9636.0,0.0,156.669189,0.000000,136.74


In [8]:
X_train_preprocessed.isna().sum()

DELIVERY_START                      0
load_forecast                      24
coal_power_available                1
gas_power_available                 1
nucelear_power_available            1
wind_power_forecasts_average       24
solar_power_forecasts_average      24
wind_power_forecasts_std           24
solar_power_forecasts_std          24
predicted_spot_price             8759
dtype: int64

In [10]:
def fill_with_local_mean(series):
    nan_indices = series[series.isna()].index  # Get indices of NaNs
    for idx in nan_indices:
        # Calculate start and end indices for the window
        start_idx = max(idx - 2, 0)
        end_idx = min(idx + 3, len(series))
        
        # Calculate mean excluding NaNs
        local_mean = series[start_idx:end_idx].mean()
        
        # Assign the local mean to the NaN value
        series.at[idx] = local_mean
        
    return series


# Exclude 'predicted_spot_price' column and fill NaNs for other columns
columns_to_fill = X_train_preprocessed.columns[X_train_preprocessed.columns != 'predicted_spot_price']
for column in columns_to_fill:
    X_train_preprocessed[column] = fill_with_local_mean(X_train_preprocessed[column])

# Check the result
print(X_train_preprocessed.isna().sum())

DELIVERY_START                      0
load_forecast                       0
coal_power_available                0
gas_power_available                 0
nucelear_power_available            0
wind_power_forecasts_average        0
solar_power_forecasts_average       0
wind_power_forecasts_std            0
solar_power_forecasts_std           0
predicted_spot_price             8759
dtype: int64


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
spot_id_delta_s = scaler.fit_transform(y_train_preprocessed["spot_id_delta"])
load_for_s = scaler.fit_transform(X_train_preprocessed["load_forecast"])
fill_values = load_for_s - spot_id_delta_s

fill_values_s = scaler.fit_transform(fill_values)
uncompleted_predicted_spot_s = scaler.fit_transform(X_train_preprocessed['predicted_spot_price'])
# Only fill NaN values in predicted_spot_price
nan_indices = uncompleted_predicted_spot_sX_train_preprocessed['predicted_spot_price'].isna()
X_train_preprocessed.loc[nan_indices, 'predicted_spot_price'] = fill_values[nan_indices]

# Check the result
print(X_train_preprocessed.isna().sum())